# Lebih Jauh tentang Aplikasi Model Neural Network

![Meme Keras](./images/meme_keras.png)

## Universal Approximation Theorem


Bunyi dari *Universal Approximation Theorem* adalah sebagai berikut:

> "Sebuah jaringan propagasi dengan hanya *hidden layer* tunggal yang memuat berhingga neuron dapat melakukan aproksimasi (pendekatan) untuk tiap fungsi kontinu dalam subset $\mathbb{R}^{n}$ ..."

**Beberapa fitur yang membuat MLP begitu baik dalam menyelesaikan masalah**:

- Stochastic Gradient Descent
- Automatic feature learning
- Klasifikasi multiclass
- Model parametrik

**Beberapa kekurangan**:

- Input harus berupa bilangan real
- Cenderung *overfitting*
- Terlalu banyak *hyperparameter* yang harus dioptimalkan
    - jumlah layer
    - fungsi aktivasi
    - konektivitas
    - inisiasi bobot
    - loss function yang digunakan
    - metode regularisasi yang digunakan
    - dll
- **Black box** *as same as* **black magic**

## Contoh Kasus: MNIST

**Load Library**

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

import matplotlib.pyplot as plt

In [ ]:
train_data = pd.read_csv("./dataset/digit_recognizer/train.csv")
test_data = pd.read_csv("./dataset/digit_recognizer/test.csv")

In [ ]:
# informasi terkait tipe dan dimensi data training
train_data.info()

In [ ]:
# informasi terkait tipe dan dimensi data test
test_data.info()

In [ ]:
# cek data yang hilang pada data training
train_data.isna().sum().describe()

In [ ]:
# cek data yang hilang pada data test
test_data.isna().sum().describe()

In [ ]:
# buat variabel untuk menyimpan kelas target
y_train = train_data['label']

y_train

In [ ]:
# buang kolom 'label' pada train data
train_data = train_data.drop(columns='label')

In [ ]:
train_data.shape

In [ ]:
# cek distribusi kelas target
y_train.value_counts().plot.barh();

In [ ]:
# grayscaling
train_data = train_data/255

test_data = test_data/255

In [ ]:
# reshaping
train_data = train_data.values

test_data = test_data.values

In [ ]:
# lihat gambar pada dataset
plt.imshow(train_data[2].reshape(28,28))
plt.show()

In [ ]:
y_train = pd.get_dummies(y_train).values

y_train

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(train_data, y_train, test_size = 0.1, random_state=1000)

In [ ]:
# Inisiasi model sequential
model = Sequential()

In [ ]:
model.add(Dense(input_dim=784, units=128, kernel_initializer='uniform', activation='sigmoid'))
model.add(Dense(units=10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=5)

In [ ]:
hasil_prediksi = model.predict_classes(X_val)

In [ ]:
y_val_sebenarnya = np.argmax(Y_val, axis=1)

In [ ]:
accuracy_score(np.argmax(Y_val, axis=1), hasil_prediksi)

In [ ]:
confusion_matrix(y_val_sebenarnya, hasil_prediksi)

**Latihan**

Coba buat model dengan tiga, empat sampai lima hidden layer. Amati apakah hasilnya semakin akurat atau tidak.

## Contoh Kasus: Time Series Forecasting

In [ ]:
dataset = pd.read_csv('./dataset/airline/airline-passengers.csv', usecols=[1], engine='python')

In [ ]:
dataset.head()

In [ ]:
plt.plot(dataset)
plt.show()

In [ ]:
dataset = dataset.values
dataset = dataset.astype('float32')

In [ ]:
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 1
trainX, trainY = create_dataset(train, time_step)
testX, testY = create_dataset(test, time_step)

In [ ]:
trainX

In [ ]:
trainY

In [ ]:
model = Sequential()
model.add(Dense(input_dim=time_step, units=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY,epochs=200)

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[time_step:len(trainPredict)+time_step, :] = trainPredict

In [ ]:
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(time_step*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
# plot baseline and predictions
plt.plot(dataset)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

**Latihan**

1. Hasilnya tidak masuk akal. Coba ganti kernel_initializer dari 'zeros' ke 'uniform' dan amati efeknya.
2. Gunakan time_step dari 2 ,3, 4, dan 5 dan amati mana yang menghasilkan output paling akurat
3. Gunakan layer dari 2, 3, dan 4 layer dan amati mana yang hasilnya paling akurat.

## Dari MLP menuju Deep Learning

Beberapa kendala dalam pemrosesan data:

- Data berukuran sangat besar
    - Jumlah baris data yang diolah
    - Jumlah fitur yang digunakan
    - Data bersifat kompleks (misalnya gambar, video, teks)

![Tantangan](./images/challenges.jpeg)

Sumber: http://cs231n.github.io/classification/

**Segmentasi Gambar**

![Bola hijau](./images/clutter.jpg)

Sumber: https://zbigatron.com/why-is-image-processing-so-hard/

**Pencahayaan**

![Pencahayaan](./images/iluminasi.png)

## Peran MLP untuk Pemrosesan Data Skala Besar

> Dengan model yang cukup kompleks dan *hidden layer* yang cukup, seharusnya MLP cukup digunakan tanpa perlu melakukan pemrosesan data lebih lanjut

![Question Mark](./images/36601.png)

Jawabannya adalah: **Tidak**!

Pada faktanya, diketahui bahwa semakin banyak **hidden layer** yang digunakan, hasilnya bisa jadi lebih buruk ketimbang model neural network yang lebih **shallow**.

![Dua Layer](./images/training_speed_2_layers.png)

![Tiga Layer](./images/training_speed_3_layers.png)

![Empat Layer](./images/training_speed_4_layers.png)

**Pengamatan**

Semakin dalam **hidden layer** dari neural network, gradient semakin kecil dan mengecil ketika melakukan propagasi balik melalui **hidden layer**. Ini artinya, *layer* yang letaknya paling awal akan belajar semakin lambat dibanding neuron yang letaknya paling depan. Fenomena ini dikatakan sebagai masalah **vanishing gradient**.

Tapi, jika yang terjadi adalah sebaliknya: justru gradient pada *layer* paling awal sangat besar dibanding *layer* yang letaknya paling depan. Hal ini disebut sebagai **exploding gradient**.

Efek dari *exploding/vanishing gradient* diantaranya adalah:

- Model memiliki nilai loss yang terlalu besar
- Hasil yang tidak stabil, dalam setiap epoch akan terjadi perubahan nilai loss yang terlalu besar
- Nilai loss dari model NaN

Cara mengetahui apakah terjadi *exploding/vanishing gradient* dengan cara berikut:

- Cek *bobot/weight* dari model
- Nilai error

Cara memperbaiki model yang mengalami masalah *exploding/vanishing gradient*:

- Gunakan layer yang lebih sedikit
- Memperkecil nilai batch
- Menggunakan model RNN seperti LSTM
- Gradient clipping (https://keras.io/optimizers/)
- Menggunakan regularizer (https://keras.io/regularizers/)